In [1]:
import torchaudio
import json
import os
from tqdm import tqdm
import math
import torch

In [2]:
from ctc_model import CausalConformerVADAdapterCTCModel
import torch
DEVICE = "cuda"
model_path = "./artifacts/ctc_vad_adapter_csj_pretrain/d0c22404e4aa4653aa3685a44b710e93/artifacts/model_20.pth"
with open(model_path, "rb") as f:
    cpt = torch.load(f)
model_state = cpt["model"]
model_args = cpt["model_args"]
model = CausalConformerVADAdapterCTCModel(**model_args).to(DEVICE)
model.load_state_dict(model_state)

ImportError: cannot import name 'CausalConformerVADAdapterCTCModel' from 'ctc_model' (/n/work3/shibutani/ASR/projects/multitask-based-adaptive-noise-reduction/ctc_model.py)

In [ ]:
from tokenizer import SentencePieceTokenizer

tokenizer = SentencePieceTokenizer(
    model_file_path="./vocabs/csj_train_nodup_sp_4096.bpe.model"
)

In [ ]:
from data import CSJVADPretrainDataset, CSJVADAdaptationDataset
"""
dev_dataset = CSJVADPretrainDataset(
    json_file_path="./json/aligned_csj/noisy_pretrain_eval_with_subsampled_vad.json",
    resampling_rate=16000,
    tokenizer=tokenizer,
    spec_aug=None,
)
"""

dev_dataset =CSJVADAdaptationDataset(
    json_file_path="./json/aligned_csj/noisy_adaptation_with_subsampled_vad.json",
    resampling_rate=16000,
    tokenizer=tokenizer,
    spec_aug=None,
)

idx = 3
_, x, y, x_len, y_len, _, _, _ = dev_dataset[idx]
bx = x.unsqueeze(0).to(DEVICE)
by = y.unsqueeze(0).to(DEVICE)
bx_len = x_len.unsqueeze(0).to(DEVICE)
by_len = y_len.unsqueeze(0).to(DEVICE)


In [ ]:
from torchmetrics.functional import char_error_rate

with torch.no_grad():
    model.eval()
    bx = bx.to(DEVICE)
    bhyp_token_ids = model.streaming_greedy_inference(bx, bx_len, 500)
    #bhyp_token_ids = model.greedy_inference(bx, bx_len)
bans_token_ids = [by[i, : by_len[i]].tolist() for i in range(by.shape[0])]
bhyp_text = tokenizer.batch_token_ids_to_text(bhyp_token_ids)
bans_text = tokenizer.batch_token_ids_to_text(bans_token_ids)
for i in range(len(bhyp_text)):
    print(bhyp_text[i])
    print(bans_text[i])
    print()
cer = char_error_rate(bhyp_text, bans_text)
print(cer)